In [113]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [5]:
# URL에서 HTML 문서 가져오기
url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
response = requests.get(url)

# BeautifulSoup 인스턴스 생성
soup = BeautifulSoup(response.text, 'html.parser')

In [26]:
# 특정 클래스를 가진 테이블 가져오기
table = soup.find('table', {
    'class': 'wikitable.sortable.sticky-header-multi.static-row-numbers.jquery-tablesorter'
})

In [28]:
# 현재 페이지에서 table 태그 모두 선택하기
table1 = soup.select('table')

In [106]:
for i in range(7):
    print(i)
    #print(table1[i])

0
1
2
3
4
5
6


In [107]:
table = table1[2]
#table

In [141]:
# 테이블 헤더 추출
headers = [th.text.strip() for th in table.find_all('th')]
headers = headers[:7]

# 테이블 행과 열 추출
rows = []
for tr in table.find_all('tr')[2:]:  # 첫 번째 행은 헤더이므로 제외
    cells = [td.text.strip() for td in tr.find_all('td')]
    #print(cells[1])
    if cells:  # 빈 행 건너뛰기
        rows.append(cells)

# 출력 확인
print("Headers:", headers)
for row in rows:  # 상위 5개 행 출력
    print(row)

Headers: ['Country/Territory', 'IMF[1][13]', 'World Bank[14]', 'United Nations[15]', 'Forecast', 'Year', 'Estimate']
['World', '115,494,312', '2025', '105,435,540', '2023', '100,834,796', '2022']
['United States', '30,337,162', '2025', '27,360,935', '2023', '25,744,100', '2022']
['China', '19,534,894', '[n 1]2025', '17,794,782', '[n 3]2023', '17,963,170', '[n 1]2022']
['Germany', '4,921,563', '2025', '4,456,081', '2023', '4,076,923', '2022']
['Japan', '4,389,326', '2025', '4,212,945', '2023', '4,232,173', '2022']
['India', '4,271,922', '2025', '3,549,919', '2023', '3,465,541', '2022']
['United Kingdom', '3,730,261', '2025', '3,340,032', '2023', '3,089,072', '2022']
['France', '3,283,429', '2025', '3,030,904', '2023', '2,775,316', '2022']
['Italy', '2,459,597', '2025', '2,254,851', '2023', '2,046,952', '2022']
['Canada', '2,330,308', '2025', '2,140,086', '2023', '2,137,939', '2022']
['Brazil', '2,307,162', '2025', '2,173,666', '2023', '1,920,095', '2022']
['Russia', '2,195,708', '2025',

In [142]:
# Pandas DataFrame으로 변환
df = pd.DataFrame(rows, columns=headers)
df = df[['Country/Territory','IMF[1][13]']][1:]

# DataFrame 출력
df.columns = ['Country','GDP']

df['GDP'] = df['GDP'].str.replace(',', '')
df['GDP'] = df['GDP'].replace('—', '0')
df['GDP'] = df['GDP'].astype(int)
df['GDP'] = df['GDP'].replace(0, np.nan)


df

,Country,GDP
1,United States,30337162.0
2,China,19534894.0
3,Germany,4921563.0
4,Japan,4389326.0
5,India,4271922.0
...,...,...
205,Kiribati,311.0
206,Palau,308.0
207,Marshall Islands,305.0
208,Nauru,161.0


In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 1 to 209
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  209 non-null    object 
 1   GDP      194 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.4+ KB


In [144]:
df = df.sort_values(by=['GDP'], ascending =[False])
df

,Country,GDP
1,United States,30337162.0
2,China,19534894.0
3,Germany,4921563.0
4,Japan,4389326.0
5,India,4271922.0
...,...,...
179,Greenland,NaN
185,Eritrea,NaN
186,Zanzibar,NaN
193,Sint Maarten,NaN


In [145]:
df['GDP'] = round(df['GDP'] * 0.001,2)
df

,Country,GDP
1,United States,30337.16
2,China,19534.89
3,Germany,4921.56
4,Japan,4389.33
5,India,4271.92
...,...,...
179,Greenland,NaN
185,Eritrea,NaN
186,Zanzibar,NaN
193,Sint Maarten,NaN


In [146]:
df.to_json('Countries_by_GDP.json')